In [6]:
import pandas as pd
import time
from pfsm import *

In [2]:
pd.set_option('display.max_colwidth', -1)
dtf_names =  pd.read_csv('data/sec_edgar_company_info.csv')
print(dtf_names.shape)
dtf_names.head()

(663000, 3)


,Line Number,Company Name,Company CIK Key
0,1,!J INC,1438823
1,2,"#1 A LIFESAFER HOLDINGS, INC.",1509607
2,3,#1 ARIZONA DISCOUNT PROPERTIES LLC,1457512
3,4,#1 PAINTBALL CORP,1433777
4,5,$ LLC,1427189


In [9]:
time_start = time.time()
M = tfidfM(dtf_names["Company Name"])
print(time.time() - time_start)

13.305208206176758


In [10]:
time_start = time.time()
# The following code runs the optimized cosine similarity function. 
# It only stores the top 10 most similar items, and only items with a similarity above 0.8
matches = awesome_cossim_top(M, M.transpose(), 10, 0.8)
print(time.time() - time_start)

2999.040043115616
